In [35]:
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
import numpy as np

In [36]:
data = open('data.txt', 'r', encoding='utf8')

In [37]:
lines = []
for i in data:
  lines.append(i)

In [39]:
data = ' '.join(lines)
data[:1000]

"\n \n \n \n                         THE ADVENTURES OF SHERLOCK HOLMES\n \n                                Arthur Conan Doyle\n \n \n \n                                 Table of contents\n \n                A Scandal in Bohemia\n                The Red-Headed League\n                A Case of Identity\n                The Boscombe Valley Mystery\n                The Five Orange Pips\n                The Man with the Twisted Lip\n                The Adventure of the Blue Carbuncle\n                The Adventure of the Speckled Band\n                The Adventure of the Engineer's Thumb\n                The Adventure of the Noble Bachelor\n                The Adventure of the Beryl Coronet\n                The Adventure of the Copper Beeches\n \n \n \n \n \n \n \n \n \n \n                               A SCANDAL IN BOHEMIA\n \n \n \n \n \n                                 Table of contents\n                                      Chapter 1\n                                      Chapter 2\n 

In [40]:
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')
data[:1000]

"                            THE ADVENTURES OF SHERLOCK HOLMES                                 Arthur Conan Doyle                                    Table of contents                 A Scandal in Bohemia                The Red-Headed League                A Case of Identity                The Boscombe Valley Mystery                The Five Orange Pips                The Man with the Twisted Lip                The Adventure of the Blue Carbuncle                The Adventure of the Speckled Band                The Adventure of the Engineer's Thumb                The Adventure of the Noble Bachelor                The Adventure of the Beryl Coronet                The Adventure of the Copper Beeches                                         A SCANDAL IN BOHEMIA                                      Table of contents                                      Chapter 1                                      Chapter 2                                      Chapter 3                          CHAPTER I     

In [41]:
data = data.split()
data = ' '.join(data)
data[:1000]

"THE ADVENTURES OF SHERLOCK HOLMES Arthur Conan Doyle Table of contents A Scandal in Bohemia The Red-Headed League A Case of Identity The Boscombe Valley Mystery The Five Orange Pips The Man with the Twisted Lip The Adventure of the Blue Carbuncle The Adventure of the Speckled Band The Adventure of the Engineer's Thumb The Adventure of the Noble Bachelor The Adventure of the Beryl Coronet The Adventure of the Copper Beeches A SCANDAL IN BOHEMIA Table of contents Chapter 1 Chapter 2 Chapter 3 CHAPTER I To Sherlock Holmes she is always the woman. I have seldom heard him mention her under any other name. In his eyes she eclipses and predominates the whole of her sex. It was not that he felt any emotion akin to love for Irene Adler. All emotions, and that one particularly, were abhorrent to his cold, precise but admirably balanced mind. He was, I take it, the most perfect reasoning and observing machine that the world has seen, but as a lover he would have placed himself in a false positio

In [44]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

In [45]:
vocab_size = len(tokenizer.word_index) + 1

In [46]:
sequence_data= tokenizer.texts_to_sequences([data])[0]

In [47]:
sequence_data[:10]

[1, 1561, 5, 129, 34, 647, 4498, 4499, 226, 5]

In [48]:
len(sequence_data)

105879

In [49]:
vocab_size

8200

In [50]:
sequence = []
for i in range(3, len(sequence_data)):
  words = sequence_data[i-3:i+1]
  sequence.append(words)

In [51]:
len(sequence)

105876

In [19]:
sequence = np.array(sequence)

In [20]:
sequence

array([[   1, 1561,    5,  129],
       [1561,    5,  129,   34],
       [   5,  129,   34,  647],
       ...,
       [  28,    1, 8198, 8199],
       [   1, 8198, 8199, 3187],
       [8198, 8199, 3187, 3186]])

In [21]:
x = []
y = []

for i in sequence:
  x.append(i[:3])
  y.append(i[3])

In [22]:
x = np.array(x)
y = np.array(y)

In [23]:
y = to_categorical(y, num_classes=vocab_size)

In [24]:
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [25]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

model.build(input_shape=(None, 3))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [26]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 3, 10)               │          82,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 3, 1000)             │       4,044,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 1000)                │       8,004,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1000)                │       1,001,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 8200)                │       8,208,200 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,339,200 (81.40 MB)

 Trainable params: 21,339,200 (81.40 MB)

 Non-trainable params: 0 (0.00 B)

In [27]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001))

In [28]:
model.fit(x, y, epochs=5, batch_size=64)

Epoch 1/5
1655/1655 ━━━━━━━━━━━━━━━━━━━━ 43s 23ms/step - loss: 6.6590
Epoch 2/5
1655/1655 ━━━━━━━━━━━━━━━━━━━━ 38s 23ms/step - loss: 5.8037
Epoch 3/5
1655/1655 ━━━━━━━━━━━━━━━━━━━━ 41s 23ms/step - loss: 5.4850
Epoch 4/5
1655/1655 ━━━━━━━━━━━━━━━━━━━━ 38s 23ms/step - loss: 5.2142
Epoch 5/5
1655/1655 ━━━━━━━━━━━━━━━━━━━━ 41s 23ms/step - loss: 4.9761


In [34]:
# Create next word prediction system
while True:

  text = input('Enter the text: ')
  if text == '1':
    print('Thank You')
    break
  else:
    text = text.split(' ')
    text = text[-3:]
    sequence = tokenizer.texts_to_sequences([text])
    sequence = np.array(sequence)
    pred = np.argmax(model.predict(sequence))
    for key, value in tokenizer.word_index.items():
      if value == pred:
        print(key)

Enter the text: THE ADVENTURES OF SHERLOCK HOLMES
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 241ms/step
was
Enter the text: 1
Thank You
